In [1]:
from pymongo import MongoClient

In [2]:
client = MongoClient('')

In [3]:
db = client.get_database('production')

In [4]:
collection = db.get_collection('orders')

In [5]:
codes = db.get_collection('zipcodes')

In [6]:
import datetime as dt

In [49]:
now = dt.datetime(2024,5,1)

In [50]:
current_month = now.month

In [93]:
records = collection.aggregate([
            {
                '$match': {
                    'address_id': {
                        '$exists': True
                    }, 
                        '$or': [
                            {
                                '$and': [
                                    {
                                        'createdOrderByShopify':  {
                                                '$eq': True
                                            }
                                    }, {
                                        'kind_of_shop': {
                                                '$eq': 'BLIFE_APP_MX'
                                            }
                                    }
                                ]
                            }, 
                            {
                            '$or': [{
                                'createdOrderByShopify': {
                                    '$exists': False
                                }
                                }, {
                                'createdOrderByShopify': {
                                '$eq': 
                                    None,
                                }
                            }], 
                            }
                        ], 
                        'paid_status': 'PAGADO'
                }
            }, {
                '$lookup': {
                    'from': 'direcciones', 
                    'localField': 'address_id', 
                    'foreignField': '_id', 
                    'as': 'address'
                }
            }, {
                '$unwind': '$address'
            }, {
                '$match': {
                    'createdAt': {
                        '$gte': dt.datetime(2024, current_month, 1, 0, 0, 0, tzinfo=dt.timezone.utc), 
                        '$lt': dt.datetime.now()
                    }
                }
            }, {
                '$project': {
                    'cp': {
                        '$convert': {
                            'input': '$address.cp', 
                            'to': 'int', 
                            'onError': '', 
                            'onNull': ''
                        }
                    }, 
                    'kind_of_shop': 1,
                    'createdOrderByShopify': 1,
                }
            }, {
                '$group': {
                    '_id': {
                        'cp': '$cp', 
                        'kind_of_shop': '$kind_of_shop',
                        'createdOrderByShopify': "$createdOrderByShopify",
                    }, 
                    'sales': {
                        '$count': {}
                    }
                }
            }, {
                '$addFields': {
                  '_id.createdOrderByShopify': {
                    '$ifNull': ['$_id.createdOrderByShopify', False]
                  }
                }
            },
            {
                '$project': {
                    'cp': '$_id.cp',
                    'kind_of_shop': '$_id.kind_of_shop',
                    'createdOrderByShopify': '$_id.createdOrderByShopify',
                    'sales': 1,
                    '_id': 0
                }
            }
        ])

In [94]:
results = list(records)

In [95]:
results

[{'sales': 1,
  'cp': 54474,
  'kind_of_shop': 'BLIFE_APP_MX',
  'createdOrderByShopify': False},
 {'sales': 1,
  'cp': 7119,
  'kind_of_shop': 'BLIFE_APP_MX',
  'createdOrderByShopify': False},
 {'sales': 1,
  'cp': 3940,
  'kind_of_shop': 'MAYOREO',
  'createdOrderByShopify': False},
 {'sales': 4,
  'cp': 64834,
  'kind_of_shop': 'BLIFE_APP_MX',
  'createdOrderByShopify': True},
 {'sales': 4,
  'cp': 55730,
  'kind_of_shop': 'BLIFE_APP_MX',
  'createdOrderByShopify': False},
 {'sales': 1,
  'cp': 60983,
  'kind_of_shop': 'BLIFE_APP_MX',
  'createdOrderByShopify': False},
 {'sales': 1,
  'cp': 96550,
  'kind_of_shop': 'BLIFE_APP_MX',
  'createdOrderByShopify': False},
 {'sales': 1,
  'cp': 34206,
  'kind_of_shop': 'BLIFE_APP_MX',
  'createdOrderByShopify': False},
 {'sales': 5,
  'cp': 52998,
  'kind_of_shop': 'BLIFE_APP_MX',
  'createdOrderByShopify': False},
 {'sales': 15,
  'cp': 77517,
  'kind_of_shop': 'BLIFE_APP_MX',
  'createdOrderByShopify': False},
 {'sales': 6,
  'cp': 2980,

In [11]:
import pandas as pd

In [54]:
from bson import ObjectId

In [77]:
df = pd.DataFrame(results)

In [78]:
df

,sales,cp,kind_of_shop,createdOrderByShopify
0,11,7790,BLIFE_APP_MX,True
1,7,24020,BLIFE_APP_MX,True
2,1,54255,BLIFE_APP_MX,False
3,7,83114,BLIFE_APP_MX,True
4,1,66458,BLIFE_APP_MX,True
...,...,...,...,...
16125,19,52978,BLIFE_APP_MX,True
16126,2,9219,BLIFE_APP_MX,True
16127,4,7740,BLIFE_APP_MX,True
16128,1,63176,BLIFE_APP_MX,True


In [14]:
zipcodes_records = codes.find()

In [15]:
df_zipcodes = pd.DataFrame(list(zipcodes_records))

In [16]:
df_zipcodes

,_id,d_codigo,D_mnpio,d_ciudad,d_estado,c_estado,d_asenta
0,66cf77f9dab1ab357d0e1b71,20000,Aguascalientes,Aguascalientes,Aguascalientes,1,[Aguascalientes Centro]
1,66cf77f9dab1ab357d0e1b72,20010,Aguascalientes,Aguascalientes,Aguascalientes,1,"[Colinas del Rio, Olivares Santana, Las Brisas..."
2,66cf77f9dab1ab357d0e1b73,20016,Aguascalientes,Aguascalientes,Aguascalientes,1,"[Colinas de San Ignacio, La Fundición, Fundici..."
3,66cf77f9dab1ab357d0e1b74,20018,Aguascalientes,Aguascalientes,Aguascalientes,1,[Línea de Fuego]
4,66cf77f9dab1ab357d0e1b75,20020,Aguascalientes,Aguascalientes,Aguascalientes,1,"[Buenos Aires, Circunvalación Norte, Las Arbol..."
...,...,...,...,...,...,...,...
32205,66cf77f9dab1ab357d0e993e,99990,Moyahua de Estrada,,Zacatecas,32,[Alameda Juárez]
32206,66cf77f9dab1ab357d0e993f,99993,Moyahua de Estrada,,Zacatecas,32,[Cuxpala]
32207,66cf77f9dab1ab357d0e9940,99994,Moyahua de Estrada,,Zacatecas,32,[Vicente Guerrero]
32208,66cf77f9dab1ab357d0e9941,99997,Moyahua de Estrada,,Zacatecas,32,[Las Palmas]


In [17]:
df_zipcodes[df_zipcodes['d_codigo'].eq(72120)]

,_id,d_codigo,D_mnpio,d_ciudad,d_estado,c_estado,d_asenta
21120,66cf77f9dab1ab357d0e6df1,72120,Puebla,Heroica Puebla de Zaragoza,Puebla,21,"[Los Arcos Santa Cruz, Ignacio Romero Vargas, ..."


In [18]:
def get_zipcode_id(row):
    res = df_zipcodes[df_zipcodes['d_codigo'].eq(row['cp'])]['_id'].tolist()
    if len(res) > 0:
        return res[0]
    print(row['cp'])
    return None

In [19]:
df_zipcodes[df_zipcodes['d_codigo'].eq(72120)]['_id']

21120    66cf77f9dab1ab357d0e6df1
Name: _id, dtype: object

In [20]:
str(df_zipcodes[df_zipcodes['d_codigo'].eq(72120)]['_id'])

'21120    66cf77f9dab1ab357d0e6df1\nName: _id, dtype: object'

In [21]:
df_zipcodes[df_zipcodes['d_codigo'].eq(72120)]['_id'].item()

ObjectId('66cf77f9dab1ab357d0e6df1')

In [22]:
str(df_zipcodes[df_zipcodes['d_codigo'].eq(72120)]['_id'].item())

'66cf77f9dab1ab357d0e6df1'

In [23]:
df_zipcodes[df_zipcodes['d_codigo'].eq(72120)]['_id'].tolist()

[ObjectId('66cf77f9dab1ab357d0e6df1')]

In [24]:
df_zipcodes[df_zipcodes['d_codigo'].eq(72120)]['_id'].tolist()[0]

ObjectId('66cf77f9dab1ab357d0e6df1')

In [25]:
df_zipcodes.loc[df_zipcodes['d_codigo'].eq(72120)]

,_id,d_codigo,D_mnpio,d_ciudad,d_estado,c_estado,d_asenta
21120,66cf77f9dab1ab357d0e6df1,72120,Puebla,Heroica Puebla de Zaragoza,Puebla,21,"[Los Arcos Santa Cruz, Ignacio Romero Vargas, ..."


In [26]:
df_zipcodes[df_zipcodes['d_codigo'].eq(72120)]['_id'].item()

ObjectId('66cf77f9dab1ab357d0e6df1')

In [18]:
df['zipcodes_id'] = df.apply(get_zipcode_id, axis=1)

5349
97248
64903
48282
54148
36251
62507

1537

7269
64833
50279
14038
80051
2450
4880
76760
22986
72216
45046
91725
62749
97294


In [94]:
df

,sales,cp,kind_of_shop,zipcodes_id
0,1,6720,BLIFE_ORDENES_MANUALES,66cf77f9dab1ab357d0e305b
1,1,64968,BLIFE_APP_MX,66cf77f9dab1ab357d0e6314
2,19,45138,BLIFE_APP_MX,66cf77f9dab1ab357d0e4746
3,1,58920,BLIFE_APP_MX,66cf77f9dab1ab357d0e58d4
4,1,57920,BLIFE_APP_MX,66cf77f9dab1ab357d0e573a
...,...,...,...,...
3522,1,70150,BLIFE_APP_MX,66cf77f9dab1ab357d0e6a3b
3523,2,54945,BLIFE_APP_MX,66cf77f9dab1ab357d0e543b
3524,2,1010,BLIFE_APP_MX,66cf77f9dab1ab357d0e2edb
3525,1,25198,BLIFE_APP_MX,66cf77f9dab1ab357d0e2151


In [20]:
def get_state(row):
    res = df_zipcodes[df_zipcodes['_id'].eq(row['zipcodes_id'])]['d_estado'].tolist()
    if len(res) > 0:
        return res[0]
    print(row['cp'])
    return None

In [21]:
df['state'] = df.apply(get_state, axis=1)

5349
97248
64903
48282
54148
36251
62507

1537

7269
64833
50279
14038
80051
2450
4880
76760
22986
72216
45046
91725
62749
97294


In [23]:
df

,sales,cp,kind_of_shop,zipcodes_id,state
0,1,59513,BLIFE_APP_MX,66cf77f9dab1ab357d0e59c0,Michoacán de Ocampo
1,1,44450,BLIFE_APP_MX,66cf77f9dab1ab357d0e4698,Jalisco
2,1,54600,MAYOREO,66cf77f9dab1ab357d0e53c7,México
3,1,63039,BLIFE_APP_MX,66cf77f9dab1ab357d0e6099,Nayarit
4,1,80110,BLIFE_APP_MX,66cf77f9dab1ab357d0e7b2e,Sinaloa
...,...,...,...,...,...
3593,1,60190,MAYOREO,66cf77f9dab1ab357d0e5ae0,Michoacán de Ocampo
3594,2,57150,BLIFE_APP_MX,66cf77f9dab1ab357d0e5706,México
3595,2,14240,BLIFE_APP_MX,66cf77f9dab1ab357d0e324e,Ciudad de México
3596,4,63195,BLIFE_APP_MX,66cf77f9dab1ab357d0e60dd,Nayarit


In [27]:
def get_state_name(row):
    res = df_zipcodes[df_zipcodes['d_codigo'].eq(row['cp'])]['d_estado'].tolist()
    if len(res) > 0:
        return res[0]
    print(row['cp'])
    return None

In [96]:
df['state_name'] = df.apply(get_state_name, axis=1)

13363
66144
66269
25205
1158
10100
98686
45132
99086
96709
53630
34078
45501
53269
62507
7457
1209
99088
88743
45302
58224
66383
61301
37735
76760
63270
86039
9205
36971
20007
33981
45418
20432
14038
72829
85428
78045
76921
98831
86116
14338
91775
36186
89128
64671
97300
50004
22570
63199
38003
60957
64969
54695
44639
22887
99059
72345
84036
91093
50989
48282
36716
55603
64208
72216
5130
86504
45206
55815
13546
63222
55771
97905
51150
86128
91725
23144
44029
36186
45538
97514
50076
44225
44949
10720
66025
90070
5210
86709
64849
78020
56668
91963
86039
83193
56180
7981
76416
24220
66175
32593
64903
44639
4220
40884
81279
36379
94292
30098
77719
20898
77636
2110
45581
48390
72821
21227
94952
2169
46610
1580
45137
28210
14130
31058
81848
41096
93821
90070
20660
7710
72485
57850
86134
53210
5300
89378
72361
97300
48274
85964
45419
16790
55874
15940
14378
32468
37135
62749
92101
29228
71234
63198
80051
48282
80129
45135
53720
91101
31128
91725
76801
54144
94662
42850
22845
65840
45409
14273

In [97]:
df

,sales,cp,kind_of_shop,createdOrderByShopify,state_name
0,11,7790,BLIFE_APP_MX,True,Ciudad de México
1,7,24020,BLIFE_APP_MX,True,Campeche
2,1,54255,BLIFE_APP_MX,False,México
3,7,83114,BLIFE_APP_MX,True,Sonora
4,1,66458,BLIFE_APP_MX,True,Nuevo León
...,...,...,...,...,...
16125,19,52978,BLIFE_APP_MX,True,México
16126,2,9219,BLIFE_APP_MX,True,None
16127,4,7740,BLIFE_APP_MX,True,Ciudad de México
16128,1,63176,BLIFE_APP_MX,True,Nayarit


In [30]:
grouped_by_state = df.groupby(df['state_name'])

In [31]:
grouped_by_state.count()

,sales,cp,kind_of_shop,createdOrderByShopify
state_name,,,,
Aguascalientes,24,24,24,15
Baja California,34,34,34,24
Baja California Sur,13,13,13,8
Campeche,11,11,11,9
Chiapas,21,21,21,14
Chihuahua,44,44,44,34
Ciudad de México,249,249,249,178
Coahuila de Zaragoza,30,30,30,24
Colima,14,14,14,13


In [98]:
grouped_by_state_kind_of_shop = df.groupby([df['state_name'], df['kind_of_shop'], df['createdOrderByShopify']])

In [99]:
grouped_by_state_kind_of_shop.value_counts()

state_name      kind_of_shop  createdOrderByShopify  sales  cp   
Aguascalientes  BLIFE_APP_MX  False                  1      20049    1
                                                            20060    1
                                                            20080    1
                                                            20089    1
                                                            20127    1
                                                                    ..
Zacatecas       MAYOREO       False                  1      99020    1
                                                            99380    1
                                                     2      98090    1
                                                            98612    1
                                                            99090    1
Name: count, Length: 15751, dtype: int64

In [100]:
grouped_by_state_kind_of_shop.size()

state_name       kind_of_shop            createdOrderByShopify
Aguascalientes   BLIFE_APP_MX            False                     97
                                         True                     139
                 BLIFE_ORDENES_MANUALES  False                      3
                 MAYOREO                 False                     14
Baja California  BLIFE_APP_MX            False                    154
                                                                 ... 
Yucatán          MAYOREO                 False                      7
Zacatecas        BLIFE_APP_MX            False                     66
                                         True                     141
                 BLIFE_ORDENES_MANUALES  False                      1
                 MAYOREO                 False                      8
Length: 119, dtype: int64

In [84]:
grouped_by_state_kind_of_shop.size().to_frame().to_records()

rec.array([('Aguascalientes', 'BLIFE_APP_MX', False,   97),
           ('Aguascalientes', 'BLIFE_APP_MX',  True,  139),
           ('Aguascalientes', 'BLIFE_ORDENES_MANUALES', False,    3),
           ('Aguascalientes', 'MAYOREO', False,   14),
           ('Baja California', 'BLIFE_APP_MX', False,  154),
           ('Baja California', 'BLIFE_APP_MX',  True,  287),
           ('Baja California', 'BLIFE_ORDENES_MANUALES', False,    6),
           ('Baja California', 'MAYOREO', False,   11),
           ('Baja California Sur', 'BLIFE_APP_MX', False,   55),
           ('Baja California Sur', 'BLIFE_APP_MX',  True,   90),
           ('Baja California Sur', 'BLIFE_ORDENES_MANUALES', False,    1),
           ('Baja California Sur', 'MAYOREO', False,    7),
           ('Campeche', 'BLIFE_APP_MX', False,   31),
           ('Campeche', 'BLIFE_APP_MX',  True,   73),
           ('Campeche', 'MAYOREO', False,    1),
           ('Chiapas', 'BLIFE_APP_MX', False,  100),
           ('Chiapas', 'BLIFE_A

In [86]:
df['state_name'].value_counts()

state_name
México                             1824
Ciudad de México                   1714
Jalisco                            1161
Veracruz de Ignacio de la Llave     948
Nuevo León                          845
Guanajuato                          817
Michoacán de Ocampo                 680
Puebla                              656
Chihuahua                           532
Sinaloa                             479
Tamaulipas                          474
Baja California                     458
Sonora                              443
Coahuila de Zaragoza                416
Hidalgo                             362
Oaxaca                              332
Morelos                             309
Querétaro                           302
Guerrero                            296
San Luis Potosí                     294
Chiapas                             281
Aguascalientes                      253
Durango                             247
Tabasco                             242
Zacatecas                    

In [87]:
df['state_name'].value_counts().to_frame().to_records()

rec.array([('México', 1824), ('Ciudad de México', 1714),
           ('Jalisco', 1161), ('Veracruz de Ignacio de la Llave',  948),
           ('Nuevo León',  845), ('Guanajuato',  817),
           ('Michoacán de Ocampo',  680), ('Puebla',  656),
           ('Chihuahua',  532), ('Sinaloa',  479), ('Tamaulipas',  474),
           ('Baja California',  458), ('Sonora',  443),
           ('Coahuila de Zaragoza',  416), ('Hidalgo',  362),
           ('Oaxaca',  332), ('Morelos',  309), ('Querétaro',  302),
           ('Guerrero',  296), ('San Luis Potosí',  294),
           ('Chiapas',  281), ('Aguascalientes',  253), ('Durango',  247),
           ('Tabasco',  242), ('Zacatecas',  216), ('Nayarit',  212),
           ('Yucatán',  196), ('Quintana Roo',  195), ('Tlaxcala',  160),
           ('Baja California Sur',  153), ('Colima',  149),
           ('Campeche',  105)],
          dtype=[('state_name', 'O'), ('count', '<i8')])

In [34]:
record_results = df['state'].value_counts().to_frame().to_records()

In [35]:
list(record_results)

[np.record(('Ciudad de México', 493), dtype=[('state', 'O'), ('count', '<i8')]),
 np.record(('México', 449), dtype=[('state', 'O'), ('count', '<i8')]),
 np.record(('Jalisco', 296), dtype=[('state', 'O'), ('count', '<i8')]),
 np.record(('Nuevo León', 192), dtype=[('state', 'O'), ('count', '<i8')]),
 np.record(('Puebla', 189), dtype=[('state', 'O'), ('count', '<i8')]),
 np.record(('Guanajuato', 173), dtype=[('state', 'O'), ('count', '<i8')]),
 np.record(('Veracruz de Ignacio de la Llave', 161), dtype=[('state', 'O'), ('count', '<i8')]),
 np.record(('Michoacán de Ocampo', 141), dtype=[('state', 'O'), ('count', '<i8')]),
 np.record(('Chihuahua', 112), dtype=[('state', 'O'), ('count', '<i8')]),
 np.record(('Sonora', 105), dtype=[('state', 'O'), ('count', '<i8')]),
 np.record(('Sinaloa', 105), dtype=[('state', 'O'), ('count', '<i8')]),
 np.record(('Baja California', 94), dtype=[('state', 'O'), ('count', '<i8')]),
 np.record(('Tamaulipas', 87), dtype=[('state', 'O'), ('count', '<i8')]),
 np.r

In [101]:
record_results = grouped_by_state_kind_of_shop.size().to_frame()

In [102]:
record_results

0
state_name      kind_of_shop           createdOrderByShopify     
Aguascalientes  BLIFE_APP_MX           False                   97
                                       True                   139
                BLIFE_ORDENES_MANUALES False                    3
                MAYOREO                False                   14
Baja California BLIFE_APP_MX           False                  154
...                                                           ...
Yucatán         MAYOREO                False                    7
Zacatecas       BLIFE_APP_MX           False                   66
                                       True                   141
                BLIFE_ORDENES_MANUALES False                    1
                MAYOREO                False                    8

[119 rows x 1 columns]

In [38]:
dict_result = record_results.to_dict('dict')

In [39]:
dict_result

{0: {('Aguascalientes', 'BLIFE_APP_MX'): 55,
  ('Aguascalientes', 'BLIFE_ORDENES_MANUALES'): 2,
  ('Aguascalientes', 'MAYOREO'): 6,
  ('Baja California', 'BLIFE_APP_MX'): 83,
  ('Baja California', 'BLIFE_ORDENES_MANUALES'): 4,
  ('Baja California', 'MAYOREO'): 7,
  ('Baja California Sur', 'BLIFE_APP_MX'): 32,
  ('Baja California Sur', 'BLIFE_ORDENES_MANUALES'): 1,
  ('Baja California Sur', 'MAYOREO'): 2,
  ('Campeche', 'BLIFE_APP_MX'): 17,
  ('Chiapas', 'BLIFE_APP_MX'): 48,
  ('Chiapas', 'BLIFE_ORDENES_MANUALES'): 2,
  ('Chiapas', 'MAYOREO'): 6,
  ('Chihuahua', 'BLIFE_APP_MX'): 100,
  ('Chihuahua', 'BLIFE_ORDENES_MANUALES'): 2,
  ('Chihuahua', 'MAYOREO'): 10,
  ('Ciudad de México', 'BLIFE_APP_MX'): 453,
  ('Ciudad de México', 'BLIFE_ORDENES_MANUALES'): 26,
  ('Ciudad de México', 'MAYOREO'): 14,
  ('Coahuila de Zaragoza', 'BLIFE_APP_MX'): 78,
  ('Coahuila de Zaragoza', 'MAYOREO'): 3,
  ('Colima', 'BLIFE_APP_MX'): 29,
  ('Colima', 'MAYOREO'): 3,
  ('Durango', 'BLIFE_APP_MX'): 34,
  ('Dur

In [103]:
count_total = grouped_by_state_kind_of_shop['state_name'].count().reset_index(name='total')

In [104]:
count_total

,state_name,kind_of_shop,createdOrderByShopify,total
0,Aguascalientes,BLIFE_APP_MX,False,97
1,Aguascalientes,BLIFE_APP_MX,True,139
2,Aguascalientes,BLIFE_ORDENES_MANUALES,False,3
3,Aguascalientes,MAYOREO,False,14
4,Baja California,BLIFE_APP_MX,False,154
...,...,...,...,...
114,Yucatán,MAYOREO,False,7
115,Zacatecas,BLIFE_APP_MX,False,66
116,Zacatecas,BLIFE_APP_MX,True,141
117,Zacatecas,BLIFE_ORDENES_MANUALES,False,1


In [105]:
count_total.to_dict('records')

[{'state_name': 'Aguascalientes',
  'kind_of_shop': 'BLIFE_APP_MX',
  'createdOrderByShopify': False,
  'total': 97},
 {'state_name': 'Aguascalientes',
  'kind_of_shop': 'BLIFE_APP_MX',
  'createdOrderByShopify': True,
  'total': 139},
 {'state_name': 'Aguascalientes',
  'kind_of_shop': 'BLIFE_ORDENES_MANUALES',
  'createdOrderByShopify': False,
  'total': 3},
 {'state_name': 'Aguascalientes',
  'kind_of_shop': 'MAYOREO',
  'createdOrderByShopify': False,
  'total': 14},
 {'state_name': 'Baja California',
  'kind_of_shop': 'BLIFE_APP_MX',
  'createdOrderByShopify': False,
  'total': 154},
 {'state_name': 'Baja California',
  'kind_of_shop': 'BLIFE_APP_MX',
  'createdOrderByShopify': True,
  'total': 287},
 {'state_name': 'Baja California',
  'kind_of_shop': 'BLIFE_ORDENES_MANUALES',
  'createdOrderByShopify': False,
  'total': 6},
 {'state_name': 'Baja California',
  'kind_of_shop': 'MAYOREO',
  'createdOrderByShopify': False,
  'total': 11},
 {'state_name': 'Baja California Sur',
  'k